In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import Label
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class SimpleImageEvaluatorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Simple Image Evaluator")
        self.root.geometry("1000x800")
        self.root.configure(bg="#E0F7FA")  # Màu nền sáng nhẹ

        self.image_path = None
        self.checkpoints = [
            'Checkpoint_FT_Aug/mobileNet_V3_Large_finetuned_final_Aug.keras',
            'Checkpoint_FT_Aug/EfficientNet_B0_finetuned_final_Aug.keras'
        ]
        self.models = [None, None]  # Danh sách để lưu các mô hình đã tải
        self.predictions = [None, None]  # Danh sách lưu dự đoán của từng mô hình

        # Left panel - Upload controls
        self.upload_frame = tk.Frame(self.root, bg="#0097A7")  # Màu xanh đậm hơn
        self.upload_frame.pack(side="left", padx=10, pady=10, fill='y')

        # Image upload button
        self.upload_image_button = tk.Button(
            self.upload_frame, text="Upload Image", command=self.select_image,
            bg="#B2EBF2", fg="#006064", font=("Arial", 14), bd=0, padx=10, pady=5
        )
        self.upload_image_button.pack(pady=5)

        # Evaluation button
        self.evaluate_button = tk.Button(
            self.upload_frame, text="Evaluate Image", command=self.evaluate_image,
            bg="#B2EBF2", fg="#006064", font=("Arial", 14), bd=0, padx=10, pady=5
        )
        self.evaluate_button.pack(pady=5)

        # Right panel - Display controls
        self.display_frame = tk.Frame(self.root, bg="#E0F7FA")
        self.display_frame.pack(side="right", padx=10, pady=10, fill='both', expand=True)

        # Image display label
        self.image_label = Label(self.display_frame)
        self.image_label.pack(expand=True)

        # Result display labels
        self.result_labels = []
        self.result_prob_labels = []
        self.results_frame = tk.Frame(self.display_frame, bg="#E0F7FA")
        self.results_frame.pack(pady=5)

        # Frame for displaying the probability charts
        self.chart_frame = tk.Frame(self.root, bg="#E0F7FA")
        self.chart_frame.pack(side="bottom", padx=10, pady=10, fill='both', expand=True)

    def select_image(self):
        file_path = filedialog.askopenfilename(
            filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp")]
        )
        if file_path:
            self.image_path = file_path
            self.display_image(file_path)

    def display_image(self, img_path):
        img = Image.open(img_path)
        img = img.resize((224, 224))  # Đảm bảo ảnh có kích thước phù hợp để hiển thị
        img = ImageTk.PhotoImage(img)
        self.image_label.configure(image=img)
        self.image_label.image = img  # Giữ tham chiếu để tránh bị xoá bởi bộ dọn rác

    def evaluate_image(self):
        if not self.image_path:
            messagebox.showerror("Error", "Please select an image.")
            return

        # Load models
        for i, checkpoint in enumerate(self.checkpoints):
            try:
                self.models[i] = load_model(checkpoint)
            except Exception as e:
                messagebox.showerror("Error", f"Failed to load model from {checkpoint}: {e}")
                return

        # Load and preprocess the image
        try:
            img = image.load_img(self.image_path, target_size=(224, 224))  # Kích thước đầu vào của mô hình
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, 0)  # Batch size = 1

            # Make predictions
            for i, model in enumerate(self.models):
                self.predictions[i] = model.predict(img_array)
                predicted_class = np.argmax(self.predictions[i], axis=1)[0]
                confidence_score = np.max(self.predictions[i])

                # Display the result
                class_names = [
                    'High lesion',
                    'Low lesion',
                    'Negative',
                    'Carcinoma'
                ]
                model_name = "Mobi V3 Large" if i == 0 else "Effi B0"
                result_label = tk.Label(
                    self.results_frame, 
                    text=f"{model_name}: {class_names[predicted_class]}",
                    font=("Arial", 14), bg="#E0F7FA", fg="#004D40"
                )
                result_label.pack(pady=5)
                self.result_labels.append(result_label)

                result_prob_label = tk.Label(
                    self.results_frame, 
                    text=f"Confidence: {confidence_score:.2f}",
                    font=("Arial", 14), bg="#E0F7FA", fg="#004D40"
                )
                result_prob_label.pack(pady=5)
                self.result_prob_labels.append(result_prob_label)

            # Call method to display the charts for both models
            self.plot_probabilities()

        except Exception as e:
            messagebox.showerror("Error", f"Error processing image: {e}")

    def plot_probabilities(self):
        # Clear previous charts
        for widget in self.chart_frame.winfo_children():
            widget.destroy()

        # Prepare data for plotting
        class_names = ['High lesion', 'Low lesion', 'Negative', 'Carcinoma']
        
        # Create figure for MobileNetV3
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.bar(class_names, self.predictions[0][0], color='#FF5733')  # MobileNetV3 Large
        ax.set_xlabel('Class')
        ax.set_ylabel('Prediction Confidence (%)')
        ax.set_title('Prediction Confidence for Mobi V3 Large')
        ax.set_ylim([0, 1])
        ax.set_xticklabels(class_names, rotation=45)
        
        # Display the first plot
        canvas = FigureCanvasTkAgg(fig, master=self.chart_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(side="top", padx=10, pady=10)  # Đặt trên cùng

        # Create figure for EfficientNetB0
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.bar(class_names, self.predictions[1][0], color='#33FF57')  # EfficientNetB0
        ax.set_xlabel('Class')
        ax.set_ylabel('Prediction Confidence (%)')
        ax.set_title('Prediction Confidence for Effi B0')
        ax.set_ylim([0, 1])
        ax.set_xticklabels(class_names, rotation=45)

        # Display the second plot
        canvas = FigureCanvasTkAgg(fig, master=self.chart_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(side="bottom", padx=10, pady=10)  # Đặt dưới cùng

# Create and run the Tkinter app
root = tk.Tk()
app = SimpleImageEvaluatorApp(root)
root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 646ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_24752\2092503231.py:147: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(class_names, rotation=45)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_24752\2092503231.py:161: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(class_names, rotation=45)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 656ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_24752\2092503231.py:147: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(class_names, rotation=45)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_24752\2092503231.py:161: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(class_names, rotation=45)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 796ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_24752\2092503231.py:147: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(class_names, rotation=45)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_24752\2092503231.py:161: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(class_names, rotation=45)


In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import Label
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class SimpleImageEvaluatorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Simple Image Evaluator")
        self.root.geometry("1000x800")
        self.root.configure(bg="#E0F7FA")

        self.image_path = None
        self.checkpoints = [
            'Checkpoint_FT_Aug/mobileNet_V3_Large_finetuned_final_Aug.keras',
            'Checkpoint_FT_Aug/EfficientNet_B0_finetuned_final_Aug.keras'
        ]
        self.models = [None, None]  # Lưu các mô hình đã tải
        self.predictions = [None, None]  # Lưu dự đoán của từng mô hình

        # Left panel - Upload controls
        self.upload_frame = tk.Frame(self.root, bg="#0097A7")
        self.upload_frame.pack(side="left", padx=10, pady=10, fill='y')

        # Image upload button
        self.upload_image_button = tk.Button(
            self.upload_frame, text="Upload Image", command=self.select_image,
            bg="#B2EBF2", fg="#006064", font=("Arial", 14), bd=0, padx=10, pady=5
        )
        self.upload_image_button.pack(pady=5)

        # Evaluation button
        self.evaluate_button = tk.Button(
            self.upload_frame, text="Evaluate Image", command=self.evaluate_image,
            bg="#B2EBF2", fg="#006064", font=("Arial", 14), bd=0, padx=10, pady=5
        )
        self.evaluate_button.pack(pady=5)

        # Right panel - Display controls
        self.display_frame = tk.Frame(self.root, bg="#E0F7FA")
        self.display_frame.pack(side="right", padx=10, pady=10, fill='both', expand=True)

        # Image display label
        self.image_label = Label(self.display_frame)
        self.image_label.pack(expand=True)

        # Frame for displaying the results
        self.results_frame = tk.Frame(self.display_frame, bg="#E0F7FA")
        self.results_frame.pack(pady=5)

        # Frame for displaying the probability charts
        self.chart_frame = tk.Frame(self.root, bg="#E0F7FA")
        self.chart_frame.pack(side="bottom", padx=10, pady=10, fill='both', expand=True)

    def select_image(self):
        file_path = filedialog.askopenfilename(
            filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp")]
        )
        if file_path:
            self.image_path = file_path
            self.display_image(file_path)

    def display_image(self, img_path):
        img = Image.open(img_path)
        img = img.resize((224, 224))
        img = ImageTk.PhotoImage(img)
        self.image_label.configure(image=img)
        self.image_label.image = img

    def clear_results(self):
        # Xóa các nhãn cũ trong `results_frame`
        for widget in self.results_frame.winfo_children():
            widget.destroy()

    def clear_charts(self):
        # Xóa các biểu đồ cũ trong `chart_frame`
        for widget in self.chart_frame.winfo_children():
            widget.destroy()

    def evaluate_image(self):
        if not self.image_path:
            messagebox.showerror("Error", "Please select an image.")
            return

        self.clear_results()
        self.clear_charts()

        # Load models
        for i, checkpoint in enumerate(self.checkpoints):
            try:
                self.models[i] = load_model(checkpoint)
            except Exception as e:
                messagebox.showerror("Error", f"Failed to load model from {checkpoint}: {e}")
                return

        # Load and preprocess the image
        try:
            img = image.load_img(self.image_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, 0)

            class_names = ['High lesion', 'Low lesion', 'Negative', 'Carcinoma']

            for i, model in enumerate(self.models):
                self.predictions[i] = model.predict(img_array)
                predicted_class = np.argmax(self.predictions[i], axis=1)[0]
                confidence_score = np.max(self.predictions[i])

                model_name = "Mobi V3 Large" if i == 0 else "Effi B0"

                # Display model name and prediction
                tk.Label(
                    self.results_frame, 
                    text=f"{model_name}: {class_names[predicted_class]}",
                    font=("Arial", 14), bg="#E0F7FA", fg="#004D40"
                ).pack(pady=5)

                tk.Label(
                    self.results_frame, 
                    text=f"Confidence: {confidence_score:.2f}",
                    font=("Arial", 14), bg="#E0F7FA", fg="#004D40"
                ).pack(pady=5)

            self.plot_probabilities(class_names)

        except Exception as e:
            messagebox.showerror("Error", f"Error processing image: {e}")

    def plot_probabilities(self, class_names):
        colors = ['#FF5733', '#33FF57']

        for i, prediction in enumerate(self.predictions):
            fig, ax = plt.subplots(figsize=(6, 4))
            ax.bar(class_names, prediction[0], color=colors[i])
            ax.set_xlabel('Class')
            ax.set_ylabel('Prediction Confidence (%)')
            ax.set_title(f'Prediction Confidence for {"Mobi V3 Large" if i == 0 else "Effi B0"}')
            ax.set_ylim([0, 1])
            ax.set_xticklabels(class_names, rotation=45)

            canvas = FigureCanvasTkAgg(fig, master=self.chart_frame)
            canvas.draw()
            canvas.get_tk_widget().pack(side="top", padx=10, pady=10)

# Create and run the Tkinter app
root = tk.Tk()
app = SimpleImageEvaluatorApp(root)
root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_16464\107412609.py:144: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(class_names, rotation=45)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_16464\107412609.py:144: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(class_names, rotation=45)
